In [3]:
%%time

#Задаем урл главной страницы категории
x = "https://gotovim-doma.ru/category/55-napitki?"

#вытаскиваем библиотеки
import requests, bs4, tqdm
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import numpy as np
from functools import reduce

#вытаскиваем html-код главной страницы раздела
def get_html(url):
    r = requests.get(url)
    return r.text

ready_html = get_html(x)

#получаем общее число страниц категории
def get_total_category_pages(ready_html):
    soup = BeautifulSoup(ready_html, 'lxml')
    divs = soup.find('div', class_='pager')
    pages = divs.find('li', class_='last')
    xxx = str(pages).split(">")[2].split("<")[0]
    return int(xxx)
#print (get_total_category_pages(ready_html))

#получаем ссылки на все страницы категории
def main_salat(base_url):
    page_part = "page="
    total_pages = get_total_category_pages(ready_html)   
    urls = []
    for i in range (1, total_pages + 1):
        url_gen = base_url + page_part + str(i)
        urls.append(url_gen)
    return urls
category_pages_url = list(main_salat(x))
#(main_salat(x)) #- формат получения записи
#print (category_pages_url)

#получаем урлы конкретных рецептов с одной страницы категории
def get_recipes_links(url):   
    ready_html = get_html(url)
    soup = BeautifulSoup(ready_html, 'lxml')
    h4 = list(soup.find_all('h4'))
    links = []
    for i in h4:
        div = i.find('a').get('href')
        url = "https://gotovim-doma.ru" + div
        links.append(url)
    return links

list_of_list_recipes = list(map(get_recipes_links, category_pages_url))
#print (list_of_list_recipes)

#делаем один массив из массива массивов
l = list_of_list_recipes
list_of_recipes = reduce(lambda x,y: x+y,l)
#print (list_of_recipes)

#получаем список ссылок уникальный рецептов со всех страниц категории
#category_url = x
#all_recipes = []
#pages = main_salat(salat_url)
#for page in tqdm_notebook(pages):
    #recipes = get_recipes_links(page)
    #for recipe in recipes:
        #all_recipes.append(recipe)
        
uniq_rec = np.unique(list_of_recipes)
#print (uniq_rec)
#len(uniq_rec)

CPU times: user 1.03 s, sys: 113 ms, total: 1.14 s
Wall time: 5.33 s


In [4]:
print (len(uniq_rec))
print (uniq_rec[1])

164
https://gotovim-doma.ru/recipe/1144-mors-iz-krasnoi-smorodiny


In [8]:
def export_recipes(url):
    s=requests.get(url)
    b=bs4.BeautifulSoup(s.text, "html.parser")
    p4=b.select('.recept-table')
    recipe1=p4[0].getText()

    from string import digits
    remove_digits = str.maketrans('', '', digits)
    res = recipe1.translate(remove_digits)
#print (res)

    res1 = res.replace("(по желанию)", "")
    
    res_list = res1.split("\n")
#print (res_list)
    black_list = ["укроп (или другая зелень)", "Для сметанного соуса с укропом и чесноком", 'полпучка', " ", " ч.л. (по вкусу)", '', " зубчик", " г", ' ст.л.', "- веточки", " полпучка", " по вкусу", " ст.л.", "  ( г)", " шт ( г)", " шт", ' я беру на глаз, так отщипну от кусочка дрожжей, примерно получается как горох', '- горсти', ' шарика', ' ст.л. (или по вкусу)', ' кружочка', '  г', ' пакетик ( г)', ' г (примерно)', ' кг', '  ст.л.', ' шт (больших)', 'по  ч.л.', ' ст.л. или корешок свежего', ' кофейные чашки (по  мл)', ' пакет', 'тоник (в идеале "белый" - schweppes, sprite, но подойдет также любая белая газированная вода)', '- ст.л.', '- ягоды', 'пол пучка ', 'для одного  мл мохито',', л + , л для ананаса', '- шт', 'пару веточек ', '- кусочка ', '- мл', 'щепотка ', ' л', 'На - стакана (в зависимости от размера стаканов)', '- г  (или по вкусу)', ', бутончика', 'вода кипяченная комнатной температуры', ', ч.л.', 'пучок эстрагона ', '-  гр.', ' кофейная чашечка', ' Гр', '- ,л ', 'хороший растворимый кофе', ', стакана', 'чеснок', '- гр.', 'сахар (*положите сразу в приготовленный кофе до того, как будете его остужать)', 'кипяток', 'г ', 'Для сиропа', '. л', ' г (~, стакана)', ' стаканов (+ еще немного (всего приблизительно пара литров)', 'по-вкусу ', ' г (в очищенном виде -  г)', 'На банку (меньше чем литровая, из-под шоколадной пасты " дней")', ' зерен', '/ шт', 'вода', 'капучино-шейк (на  порцию)', ', стакан', '- тонких пластинки','на одну порцию', 'Либо ягоды и фрукты', ' шт (или ананасовый сок  мл)', ' г ( пакетик)', '/ палочки', ' шт (или лайм -  шт. вкус будет совершенно другим)', '/ зубчика', '- стебля', ' u', '- г (- ст.л.)', 'Для коктейля', "изюм (для подачи, по желанию)", ' ст.л. (с горкой)',  '+ л', '-  ст.л.', ' чашки', 'по вкусу ', 'сахар по вкусу (можно и без сахара)', '- шт. ', ', ст.л.', ' частей', 'лёд', ' листов (крупных)', ' пачки по  мл', ' гр (или по вкусу)', ' пакетик', 'щепотка корицы по вкусу', ' кг (вес без косточек)','/-/ стакана', 'вода (кипячёная)', ', шт', 'терпение!', ' г (уже очищенная мякоть)', ', литра', ' ч.л. (или по вкусу)', 'корочки - фруктов (апельсины, мандарины, лимоны, грейпфруты)', 'лимоны с тонкой шкуркой (можно чередовать: лимон - апельсин, лимон - лайм, апельсин - грейпфрут)', 'лед (много)', 'клубника (замороженная) ', 'овсяные хлопья - - ст.ложки с небольшой горкой', ',  ст.л.', ', кг', '- палочки', ' палочки', ' ломтик', '- ч.л.', 'минеральная вода (холодная)', ' ст.л.( или по вкусу)','- ст.л.(по вкусу)', 'клубника, мята - для украшения', '- стеблей ', '- бутона', ' шт.', ' шт спелые или  очень большой', 'соль, сахар по вкусу', 'холодная не газированая вода', 'вода, молоко или сливки', 'вода кипяченая (очищенная)', ' см', ' ч.л.', 'половинка ', 'вода (для чая)', '- г', 'на  чашки', '  ст.л. с горкой (примерно  гр)', 'маленькая щепотка соли', '- листика', 'для миндального молока', ',- стакана', ' листиков', ' звездочки', 'арбуз, любой (можно и не сладкий, который кушать не очень хочется)', ". ч.л.", ' мл для спирта (или  мл для водки)', 'ананас для подачи', 'цедра и сок двух лимонов', ' пару кубиков', 'холодная вода', '-, г', 'Для смузи', ' звёздочек ', 'лавровых ', 'сахар (мелкий)', '/ ст.л.', 'на  л воды', ' пачки (х, л)', 'минеральная газированная вода', ' баночка', '. ст.л', '- шт (в зависимости от их размера)', 'кусочек в  см ', '- коробочек', 'вода (дополнительно, для растворения крахмала)', 'горсть миндальных орехов (не указываю точное колличество)', ' ломтей', ' банка ( мл)', '"чудо - молоко" шоколадное (я в молоко добавила  ст.л. какао)', ' (большой (примерно  гр)', ' палочка ', '-, литра', 'заварной чай', ' на кончике ножа', '*подробно см рецепт', '(сахар и ром можно добавлять по вкусу)', 'для ', ' листочков', '-, л', 'для тостов', ' мл. очищенного ', '. кг', 'пол-лимона', 'банка сгущенки', 'горячий кофе', ' долька', ', ч.л. (я использовала ванильный сахар)', 'для  литра мацуна', 'вода кипяченая', '- г (я добавляла коньяк  г) мне нравится нежный', 'холодное цельное молоко', ' стручок', 'ёмкость (например, банка)', 'небольшой кусочек имбиря (кто не любит имбирь, можно не доабвлять)', ' части', 'от половинки лимона', '*сахара можно добавлять и больше, т.к. получается не очень сладко', 'цедра половины лимона', 'пол ч.л. сушёного молотого имбиря (я заменила на кусочек свежего  см)', ' стакан', 'Для напитков', '- г ( добавляйте по вкусу)', ' на кончике чайной ложки', ' банка', 'сахар коричневый (не добавляла)', '/ ст', ' листьев', ' ч.л. с горкой (я сварила  г натурального крепкого кофе)', 'кубики льда (или можно использовать кубики замороженного молока)', 'несколько кубиков льда', 'соль', ' по вкусу (я не добавляла)', 'на  чайник , л -  порции', ' пачки (х г)', 'для тыквенного молока', ' ст.л. с горкой ( - г)', 'яблочный сок или молоко, чтобы разбавить смузи до нужной густоты', ' гр.', ' г для спирта (или  г для водки)', ' мл  + ещё примерно - мл (см.рецепт)', '  ч.л.', ' литр', 'лед', ' чашечки (или  ч. л. растворимого кофе)', ' стакана', 'вода холодная', ' несколько', 'колотый лед', 'кг шт', 'для ласси', 'сильногазированная питьевая вода (или содовая)', ' кусочек (примерно - см)', 'Для сахарного сиропа:', ' по вкусу и по желанию', '- шт (маленькие)', ' полстакана', '/ ч.л.', '- веточек ', 'традиционное соотношение чая и кофе в этом напитке', 'состав (на л)',' щепотка', ' ст.л. без горки', 'лимонная кислота на кончике ножа', ',- ч.л.', 'кубики льда', ' вариант клубничный ликер', 'вода (или фруктовые соки)', ' гр', ' Г', 'для  ', ', л', ' мл', '*с таким количеством крахмала кисель получается очень густой, если хотите более жидкий, то можно уме', 'на  - х литровую банку', ' бутылка', 'для напитка', 'вода питьевая', 'для морса', 'Для коктейля:', ' мл ', ' шт. ', 'Сахар (по вкусу)', ' мл (или водка -  мл)', 'вода для заваривания', 'шоколадное чудо-молоко (это какао уже разведеный, в пачках)', ' щепотка по желанию', 'для овсяного молока', ' литра', 'Для кваса', 'вода для разбавления (минеральная с газом или охлажденная кипяченая)', '-  шт', 'скорлупа кедровых орехов', 'для сервировки - киви, листья мяты', 'мед', '   ', '/ стакана', 'дюжина или около того маленьких листиков мяты (свежих)', 'по желанию ']
    l3 = [x for x in res_list if x not in black_list]
    #l1 = res_list
    #l2 = ["укроп (или другая зелень)", "Для сметанного соуса с укропом и чесноком", 'полпучка', " ", " ч.л. (по вкусу)", '', " зубчик", " г", ' ст.л.'
     #" по вкусу", "- веточки", " полпучка", " по вкусу", " ст.л.", "  ( г)", " шт ( г)", " шт"]

    #l3 = [x for x in l1 if x not in l2]
#print (l3)
    return l3

    #res_set = set(l3)
    #return (res_set)

list_of_list_ingr = list(map(export_recipes, uniq_rec))

In [ ]:
cool_words = ["сок", "кофе", "лимоны"]
words = {
    'sok': 'sok',
    'soki': 'sok',
    'limon': 'limon',
        ''
}
list_of_ingr_un = list(set_of_ingr)
for i in range(len(list_of_ingr_un)):
    for word in cool_words:
        if word in list_of_ingr_un[i]:
            list_of_ingr_un[i] = word
            break

print (list_of_ingr_un)

In [9]:
#print (list_of_list_ingr)
l1 = list_of_list_ingr
list_of_ingr = reduce(lambda x,y: x+y,l1)
#print (list_of_ingr)
set_of_ingr = set(list_of_ingr)
print (set_of_ingr)

{'красная смородина (чистый вес ягод)', 'ягоды (у меня замороженные малина и клубника)', 'йогурт', 'корень имбиря', 'какао-порошок', 'горький или темный шоколад', 'сода ', 'яблок', 'корица молотая ', 'мёд', 'зелёные яблоки', 'цикорий молотый', 'Водка - об.%', 'сметана', 'кофе растворимsq (я варила кофе из молотых кофейных зерен)', 'джем низкокалорийный', 'Сок половины апельсина', 'хлеб ржаной', 'белое сухое вино (хорошее)', 'курага', 'сок гранатовый', 'ваниль', ' свёкла (средняя)', 'сахар коричневый (тростниковый) ', 'желток яичный', 'горошины чёрного перца', 'банан (очень спелый)', 'лимоны (крупные)', 'сливки  %', 'гвоздика', 'шоколад', 'вода ', 'крепкий холодный кофе', 'апельсины', 'персик греческий', 'сахар или мёд', 'свежий тархун (эстрагон)', 'кардамон зеленый', 'клубника (чем вкуснее клубника - тем вкуснее десерт)', 'пол горсти изюма (лучше светлого)', 'арбуз очищенный ', 'звёздочки бадьяна', 'кардамон', 'Мятно-овощной ласси', 'дрожжи сухие', 'помидор', 'ликер kahlua (или другой 

In [36]:
cool_words = ["сок", "кофе", "лимоны"]
words = {
    'ягод': 'ягоды',
    'ягоды': 'ягоды',
    'limon': 'limon',
        ''           
        }
list_of_ingr_un = list(set_of_ingr)
for i in range(len(list_of_ingr_un)):
    for word in words:
        if word in list_of_ingr_un[i]:
            list_of_ingr_un[i] = word
            break

print (list_of_ingr_un)

SyntaxError: invalid syntax (<ipython-input-36-1c6ff8822097>, line 7)

In [37]:
from tqdm import tqdm_notebook